In [1]:
#    ____       _       _
#   |  _ \  ___| |_ __ | |__   __ _  ___
#   | | | |/ _ \ | '_ \| '_ \ / _` |/ _ \
#   | |_| |  __/ | |_) | | | | (_| |  __/
#   |____/ \___|_| .__/|_| |_|\__,_|\___|
#                |_|

version 1.6
<br>
date 2019-08-15
<br>
author: Carlo van Wordragen

In [31]:
import requests
import json
from datetime import datetime as dt
import pandas as pd

## Azure AD Authentication

https://docs.microsoft.com/nl-nl/azure/active-directory/develop/howto-create-service-principal-portal

1. create Azure AD application in 'App registrations' <br>
    for example: 'azure-lab4-arm-api' <br>
    redirect: http://localhost <br>
    create 'Client secret' and save to variable azure_secret <br>
<br>    
2. assign application to a RBAC role 'Owner'<br>
   via Cost Management + Billing, select Subscription 'Free Trial'<br>
   Access Control (IAM)<br>
   Role assignments<br>
   Search by application name 'azure-lab4-arm-api'<br>
   Add role assigment to 'Owner'
   

### Azure ID's

In [32]:
# azure-lab4-arm-api

# subscription ID can be found on a resource, overview page
azure_subscription_id = 'a8682560-0e6b-4834-accf-21ff95b29125'

# app information can be found:
# Azure Active Directory
# app Registrations
# overview
azure_tenant = '3089f1b6-d716-42a9-9dd5-a580cf7b4f12'
azure_client_id = '0407876f-fcd1-4260-bfa4-4180a54beced' # application ID

# client secret only visible during creation
# copy & pase here
azure_secret = 'iV-6G@Ee=1Cw7T/C0G1RPkPP/S[W/D+r'

### Request Access Token

In [33]:
# authorize with azure

#
# tutorial
# http://exponentialcoding.com/accessing-azure-rest-api-from-python/
#
# access_token expires in 1 hour
#

url = "https://login.windows.net/%s/oauth2/token" % azure_tenant

#resource = 'https://management.core.windows.net' # Azure Classic Deployment
resource = 'https://management.azure.com' # Azure Resource Manager
parameters = {
    'resource' : resource,
    'client_id' : azure_client_id,
    'grant_type' : 'client_credentials',
    'client_secret': azure_secret
}

headers = {'Content-Type': 'application/x-www-form-urlencoded'}
response = requests.post(url, data=parameters, headers=headers)
jsdata = response.json()

print (json.dumps(jsdata, indent=4))
access_token = jsdata['access_token']

{
    "token_type": "Bearer",
    "not_before": "1565858400",
    "access_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCIsImtpZCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMzA4OWYxYjYtZDcxNi00MmE5LTlkZDUtYTU4MGNmN2I0ZjEyLyIsImlhdCI6MTU2NTg1ODQwMCwibmJmIjoxNTY1ODU4NDAwLCJleHAiOjE1NjU4NjIzMDAsImFpbyI6IjQyRmdZSmk1NzlmcFUxZFVnejZITDJoYk5QMVhQQUE9IiwiYXBwaWQiOiIwNDA3ODc2Zi1mY2QxLTQyNjAtYmZhNC00MTgwYTU0YmVjZWQiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8zMDg5ZjFiNi1kNzE2LTQyYTktOWRkNS1hNTgwY2Y3YjRmMTIvIiwib2lkIjoiNjA0NTZjNDMtMjg2OS00Njk1LTlhMzktNzAzZTFmMzU3OWIxIiwic3ViIjoiNjA0NTZjNDMtMjg2OS00Njk1LTlhMzktNzAzZTFmMzU3OWIxIiwidGlkIjoiMzA4OWYxYjYtZDcxNi00MmE5LTlkZDUtYTU4MGNmN2I0ZjEyIiwidXRpIjoiSWp3N09JLWNZMEtEc00xSU44dnNBQSIsInZlciI6IjEuMCJ9.IcvJB6EWCyx6v1F5dnKlFzY4qowlTguT0Dt_Jaez81JIDRxL0pZ5yuXtaiAwqB4EXM1isTZZ6980dhJwgopQr2Hnj4mStjr7sE1Dyf6

In [35]:
# access token valid for 1 hour
# access token experation time
UTC = 2
print ('GMT %s' % (dt.fromtimestamp(float(jsdata['expires_on']))))
print ('CET %s' % (dt.fromtimestamp(float(jsdata['expires_on'])+UTC*3600)))

GMT 2019-08-15 09:45:00
CET 2019-08-15 11:45:00


## Azure Resource Explorer
https://resources.azure.com

copy & paste the URL to variable 'url'<br>
your Azure subscription_id will change after the Free Trial period<br>
copy & past the new URL and save to variable 'url' below

In [36]:
headers = {'Authorization': 'Bearer ' + access_token, 
           'Content-Type': 'application/json'}

In [41]:
# subscriptions

url = 'https://management.azure.com/subscriptions?api-version=2014-04-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))

jsdata['value'][0]['subscriptionId'] == azure_subscription_id

200
[
    {
        "subscriptionPolicies": {
            "locationPlacementId": "Public_2014-09-01",
            "quotaId": "FreeTrial_2014-09-01",
            "spendingLimit": "On"
        },
        "subscriptionId": "a8682560-0e6b-4834-accf-21ff95b29125",
        "state": "Enabled",
        "displayName": "Free Trial",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125"
    }
]


True

In [43]:
# resource groups

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))

200
[
    {
        "properties": {
            "provisioningState": "Succeeded"
        },
        "location": "westeurope",
        "name": "cloud-shell-storage-westeurope",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/cloud-shell-storage-westeurope"
    },
    {
        "properties": {
            "provisioningState": "Succeeded"
        },
        "location": "westeurope",
        "name": "delphae-freetrail",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail"
    },
    {
        "properties": {
            "provisioningState": "Succeeded"
        },
        "location": "westeurope",
        "name": "NetworkWatcherRG",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/NetworkWatcherRG"
    }
]


In [44]:
# delphae-freetrail

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail?api-version=2014-04-01"
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata, indent=4))

200
{
    "properties": {
        "provisioningState": "Succeeded"
    },
    "location": "westeurope",
    "name": "delphae-freetrail",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail"
}


In [45]:
# virtual machines

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Compute/virtualMachines?api-version=2018-10-01'
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'][0], indent=4))

for vm in jsdata['value']:
    print ('{:20s}{:15s}{:15s}{:10s}'.format(vm['name'], 
                           vm['location'], 
                           vm['properties']['storageProfile']['osDisk']['osType'], 
                           vm['properties']['storageProfile']['imageReference']['sku']))

200
{
    "type": "Microsoft.Compute/virtualMachines",
    "location": "northeurope",
    "properties": {
        "diagnosticsProfile": {
            "bootDiagnostics": {
                "storageUri": "https://delphaefreetraildiag131.blob.core.windows.net/",
                "enabled": true
            }
        },
        "osProfile": {
            "linuxConfiguration": {
                "provisionVMAgent": true,
                "disablePasswordAuthentication": false
            },
            "adminUsername": "cvwordragen",
            "requireGuestProvisionSignal": true,
            "computerName": "azure-ie-ubuntu-03",
            "allowExtensionOperations": true,
            "secrets": []
        },
        "provisioningState": "Succeeded",
        "networkProfile": {
            "networkInterfaces": [
                {
                    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/networkInterfaces/azure-

In [47]:
# connections

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
#print (json.dumps(jsdata['value'][0], indent=4))
print (json.dumps(jsdata['value'][0]['properties'], indent=4))

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections/lab2-meraki-vpn?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()

200
{
    "connectionType": "IPsec",
    "localNetworkGateway2": {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan"
    },
    "resourceGuid": "28241ef5-734f-42d3-bf4b-8b31c624672d",
    "routingWeight": 0,
    "ingressBytesTransferred": 0,
    "expressRouteGatewayBypass": false,
    "virtualNetworkGateway1": {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/virtualNetworkGateways/lab2-vnet-gw"
    },
    "egressBytesTransferred": 0,
    "connectionProtocol": "IKEv1",
    "ipsecPolicies": [],
    "usePolicyBasedTrafficSelectors": false,
    "enableBgp": false,
    "provisioningState": "Succeeded"
}
200


In [48]:
print (json.dumps(jsdata, indent=4))

{
    "type": "Microsoft.Network/connections",
    "location": "westeurope",
    "name": "lab2-meraki-vpn",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections/lab2-meraki-vpn",
    "properties": {
        "sharedKey": "Delphae2019",
        "connectionStatus": "Connected",
        "connectionType": "IPsec",
        "ingressBytesTransferred": 697416,
        "resourceGuid": "28241ef5-734f-42d3-bf4b-8b31c624672d",
        "routingWeight": 0,
        "localNetworkGateway2": {
            "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan"
        },
        "expressRouteGatewayBypass": false,
        "virtualNetworkGateway1": {
            "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/virtualNetworkGateways/lab2-vnet-gw"
     

In [49]:
properties = jsdata['properties']
print (properties['connectionStatus'])
print (properties['ingressBytesTransferred'] / 1024.0 / 1024.0)
print (properties['egressBytesTransferred'] / 1024.0 / 1024.0)

Connected
0.6651077270507812
5.562522888183594


In [52]:
# localNetworkGateways

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'][0], indent=4))


200
{
    "type": "Microsoft.Network/localNetworkGateways",
    "location": "westeurope",
    "name": "lab2-meraki-sdwan",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan",
    "properties": {
        "provisioningState": "Succeeded",
        "resourceGuid": "3e7c0b0a-6e51-417a-8b47-bbe8b8dd8d27",
        "localNetworkAddressSpace": {
            "addressPrefixes": [
                "192.168.64.0/24"
            ]
        },
        "gatewayIpAddress": "84.31.151.136"
    },
    "etag": "W/\"191f5411-3325-4403-bd60-670914c7bd61\""
}


In [53]:
localgw = jsdata['value'][0]
print ('{:20}{}'.format(localgw['name'], localgw['properties']['localNetworkAddressSpace']))
print ('{:20}{}'.format(localgw['name'], localgw['properties']['gatewayIpAddress']))

lab2-meraki-sdwan   {'addressPrefixes': ['192.168.64.0/24']}
lab2-meraki-sdwan   84.31.151.136


In [54]:
# resources

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resources?api-version=2017-05-10'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

df = pd.DataFrame(data)

200


In [55]:
df[['name','type','location']][df.type=='Microsoft.Compute/virtualMachines']

,name,type,location
6,azure-fr-ubuntu-07,Microsoft.Compute/virtualMachines,francecentral
7,azure-ie-ubuntu-03,Microsoft.Compute/virtualMachines,northeurope
8,azure-nl-ubuntu-01,Microsoft.Compute/virtualMachines,westeurope
9,azure-uk-ubuntu-04,Microsoft.Compute/virtualMachines,ukwest


In [56]:
df[['name','type','location']][df.type=='Microsoft.Network/localNetworkGateways']

,name,type,location
15,lab2-meraki-sdwan,Microsoft.Network/localNetworkGateways,westeurope


In [57]:
df[['name','type','location']][df.type=='Microsoft.Network/virtualNetworkGateways']

,name,type,location
30,lab2-vnet-gw,Microsoft.Network/virtualNetworkGateways,westeurope


In [58]:
df[['name','type','location']][df.type=='Microsoft.Network/connections']

,name,type,location
14,lab2-meraki-vpn,Microsoft.Network/connections,westeurope


In [59]:
# public IP addresses

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/publicIPAddresses?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

for item in data:
    try:
        properties = item['properties']
        print ('{:25s}{:15s}{}'.format(item['name'], item['location'], properties['ipAddress']))
    except:
        pass

200
lab2-vnet-ip             westeurope     52.178.29.168


In [60]:
# virtual networks

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/providers/Microsoft.Network/virtualNetworks?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']
#print (json.dumps(data[0], indent=4))


for virtnet in data:
    properties = virtnet['properties']
    print ('virtualnet: {:20}{}'.format(virtnet['name'], properties['addressSpace']['addressPrefixes'][0]))
    subnets = properties['subnets']
    for subnet in subnets:
        print ('subnet: {:24}{}'.format(subnet['name'], subnet['properties']['addressPrefix']))
    print ()

200
virtualnet: azure-ie-vnet-01    10.1.0.0/16
subnet: azure-ie-subnet-01      10.1.0.0/24
subnet: GatewaySubnet           10.1.1.0/24

virtualnet: labsource-vnet      10.3.1.0/24
subnet: labsubnet               10.3.1.0/25
subnet: GatewaySubnet           10.3.1.128/28

virtualnet: azure-uk-vnet-04    10.8.0.0/16
subnet: azure-uk-subnet-06      10.8.0.0/24
subnet: GatewaySubnet           10.8.1.0/24

virtualnet: azure-fr-vnet-08    10.5.0.0/16
subnet: azure-fr-subnet-07      10.5.0.0/24



### Create a Resoure Group

In [61]:
import numpy as np

In [62]:
resource_group_name = 'azure-lab4-apitest-%03d' % np.random.randint(100)

url = "https://management.azure.com/subscriptions/" + azure_subscription_id + "/resourcegroups/" + resource_group_name + "?api-version=2017-05-10"
headers = { 'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json' }
body = { 'name': resource_group_name, 'location': 'westeurope' }
response = requests.put(url, data=json.dumps(body), headers=headers)
print (response.status_code)


print (json.dumps(response.json(), indent=4))

201
{
    "properties": {
        "provisioningState": "Succeeded"
    },
    "location": "westeurope",
    "name": "azure-lab4-apitest-043",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/azure-lab4-apitest-043"
}


In [63]:
# resource groups

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
if response.status_code!=200:
    print (response.text)
jsdata = response.json()
#print (json.dumps(jsdata['value'], indent=4))
pd.DataFrame(jsdata['value'])[['name','location']]

200


,name,location
0,azure-lab4-apitest-043,westeurope
1,cloud-shell-storage-westeurope,westeurope
2,delphae-freetrail,westeurope
3,NetworkWatcherRG,westeurope


### Delete a Resoure Group

In [64]:
#resource_group_name = 'azure-lab4-apitest-044' 
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/' + resource_group_name + '?api-version=2014-04-01'
headers = { 'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json' }
#body = { 'name': resource_group_name, 'location': 'westeurope' }
body = { 'name': resource_group_name}
response = requests.delete(url, data=json.dumps(body), headers=headers)
print (response.status_code)
print (response.text)

202



In [65]:
# ProvisioningState: Deleting

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))
#pd.DataFrame(jsdata['value'])[['name','location']]

200
[
    {
        "properties": {
            "provisioningState": "Deleting"
        },
        "location": "westeurope",
        "name": "azure-lab4-apitest-043",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/azure-lab4-apitest-043"
    },
    {
        "properties": {
            "provisioningState": "Succeeded"
        },
        "location": "westeurope",
        "name": "cloud-shell-storage-westeurope",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/cloud-shell-storage-westeurope"
    },
    {
        "properties": {
            "provisioningState": "Succeeded"
        },
        "location": "westeurope",
        "name": "delphae-freetrail",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail"
    },
    {
        "properties": {
            "provisioningState": "Succeeded"
        },
        "location": "westeurope",
        "name": "NetworkWatcherRG",
        

### Express Route Service Providers (ERSP)

In [66]:
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/providers/Microsoft.Network/expressRouteServiceProviders?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

200


In [67]:
for ersp in data:
    if ersp['name']=='Equinix':
        print (json.dumps(ersp, indent=4))

{
    "type": "Microsoft.Network/expressRouteServiceProviders",
    "properties": {
        "bandwidthsOffered": [
            {
                "valueInMbps": 50,
                "offerName": "50Mbps"
            },
            {
                "valueInMbps": 100,
                "offerName": "100Mbps"
            },
            {
                "valueInMbps": 200,
                "offerName": "200Mbps"
            },
            {
                "valueInMbps": 500,
                "offerName": "500Mbps"
            },
            {
                "valueInMbps": 1000,
                "offerName": "1Gbps"
            },
            {
                "valueInMbps": 2000,
                "offerName": "2Gbps"
            },
            {
                "valueInMbps": 5000,
                "offerName": "5Gbps"
            },
            {
                "valueInMbps": 10000,
                "offerName": "10Gbps"
            }
        ],
        "provisioningState": "Succeeded",
     

In [68]:
for ersp in data:
    if 'Amsterdam' in ersp['properties']['peeringLocations']:
        #print (json.dumps(ersp, indent=4))
        values = [x['valueInMbps'] for x in ersp['properties']['bandwidthsOffered']]
        print ('{:40}{}'.format(ersp['name'], values))

Aryaka Networks                         [50, 100, 200, 500, 1000, 2000, 5000, 10000]
AT&T Netbond                            [50, 100, 200, 500, 1000, 2000, 5000, 10000]
British Telecom                         [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Colt Ethernet                           [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Colt IPVPN                              [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Equinix                                 [50, 100, 200, 500, 1000, 2000, 5000, 10000]
euNetworks                              [50, 100, 200, 500, 1000, 2000, 5000, 10000]
GEANT                                   [50, 100, 200, 500, 1000, 2000, 5000, 10000]
InterCloud                              [50, 100, 200, 500, 1000, 2000, 5000, 10000]
InterCloud for Azure                    [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Interxion                               [50, 100, 200, 500, 1000, 2000, 5000, 10000]
IX Reach                                [50, 100, 200, 500, 1000,